### Movenet test 

In [56]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed 
import numpy as np
import cv2
import os
import keras
from sklearn.model_selection import train_test_split


# Matplotlib 
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
import matplotlib.patches as patches

# Animation libraries 
import imageio
from IPython.display import HTML, display

import os
import numpy as np
from PIL import Image

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv2D, MaxPooling2D, LSTM
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator



### Load movenet model from TF Hub

In [46]:
model_name = "movenet_lightning"

if "tflite" in model_name:
  if "movenet_lightning_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/float16/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_f16" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/float16/4?lite-format=tflite
    input_size = 256
  elif "movenet_lightning_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/lightning/tflite/int8/4?lite-format=tflite
    input_size = 192
  elif "movenet_thunder_int8" in model_name:
    !wget -q -O model.tflite https://tfhub.dev/google/lite-model/movenet/singlepose/thunder/tflite/int8/4?lite-format=tflite
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  # Initialize the TFLite interpreter
  interpreter = tf.lite.Interpreter(model_path="model.tflite")
  interpreter.allocate_tensors()

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    # TF Lite format expects tensor type of uint8.
    input_image = tf.cast(input_image, dtype=tf.uint8)
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    interpreter.set_tensor(input_details[0]['index'], input_image.numpy())
    # Invoke inference.
    interpreter.invoke()
    # Get the model prediction.
    keypoints_with_scores = interpreter.get_tensor(output_details[0]['index'])
    return keypoints_with_scores

else:
  if "movenet_lightning" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/lightning/4")
    input_size = 192
  elif "movenet_thunder" in model_name:
    module = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    input_size = 256
  else:
    raise ValueError("Unsupported model name: %s" % model_name)

  def movenet(input_image):
    """Runs detection on an input image.

    Args:
      input_image: A [1, height, width, 3] tensor represents the input image
        pixels. Note that the height/width should already be resized and match the
        expected input resolution of the model before passing into this function.

    Returns:
      A [1, 1, 17, 3] float numpy array representing the predicted keypoint
      coordinates and scores.
    """
    model = module.signatures['serving_default']

    # SavedModel format expects tensor type of int32.
    input_image = tf.cast(input_image, dtype=tf.int32)
    # Run model inference.
    outputs = model(input_image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints_with_scores = outputs['output_0'].numpy()
    return keypoints_with_scores

### Load Images from Folders 

In [47]:
def load_all_images():
    # Root directory containing all numbered folders
    root_dir = os.path.join(os.getcwd(), 'resized_images')
    
    # Number of folders (0-15)
    num_folders = 16
    
    # Create a single list to store all images
    all_images = []
    
    # Loop through each folder
    for folder_num in range(num_folders):
        folder_path = os.path.join(root_dir, str(folder_num))
        
        # Check if folder exists
        if not os.path.exists(folder_path):
            print(f"Warning: Folder {folder_path} does not exist.")
            continue
            
        # Get all image files from the folder
        image_files = [f for f in os.listdir(folder_path) 
                      if f.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp'))]
        
        # Load each image
        for img_file in image_files:
            img_path = os.path.join(folder_path, img_file)
            try:
                # Open the image
                img = Image.open(img_path)
                
                # Convert to numpy array
                img_array = np.array(img)
                
                # Add to the single list
                all_images.append(img_array)
            except Exception as e:
                print(f"Error loading {img_path}: {e}")
    
    print(f"Total images loaded: {len(all_images)}")
    return all_images

# Load all images into a single array
images = load_all_images()

images

Total images loaded: 192


[array([[[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
  

### Apply Movenet to images to get the key points 

In [48]:
# Resize and pad the image to keep the aspect ratio and fit the expected size.

def keypoints(images):
  keypoints_with_scores = []

  for image in images:
    input_image = tf.expand_dims(image, axis=0)
    input_image = tf.image.resize_with_pad(input_image, input_size, input_size)

    keypoints_with_scores.append(movenet(input_image))


  return keypoints_with_scores


movenet_keypoints = keypoints(images=images)

movenet_keypoints



[array([[[[0.1425851 , 0.46509394, 0.69782394],
          [0.1174522 , 0.4941325 , 0.82675546],
          [0.11353093, 0.4383859 , 0.69209206],
          [0.13424051, 0.52769554, 0.8627108 ],
          [0.13186938, 0.39917207, 0.7293694 ],
          [0.2683382 , 0.60730314, 0.8329263 ],
          [0.30618325, 0.3285296 , 0.7674295 ],
          [0.5046009 , 0.7016752 , 0.9048787 ],
          [0.55841446, 0.27859393, 0.9433538 ],
          [0.38060582, 0.645345  , 0.79218405],
          [0.41102645, 0.32034194, 0.8586101 ],
          [0.7260906 , 0.5667174 , 0.84936166],
          [0.73006594, 0.39637858, 0.80705225],
          [0.96690536, 0.6057162 , 0.41834345],
          [0.9914292 , 0.3724566 , 0.49955344],
          [0.8863649 , 0.58780026, 0.07404406],
          [0.9344226 , 0.37633014, 0.11465928]]]], dtype=float32),
 array([[[[0.16157779, 0.49504828, 0.7726422 ],
          [0.1339223 , 0.52195716, 0.5970783 ],
          [0.13508539, 0.4649422 , 0.9064761 ],
          [0.15025498

### Convert to numPy array and remove unnessasary dimensions

In [60]:
keypoints_array = np.array(movenet_keypoints)
keypoints_array = keypoints_array.squeeze(axis=(1,2))

keypoints_array


array([[[0.1425851 , 0.46509394, 0.69782394],
        [0.1174522 , 0.4941325 , 0.82675546],
        [0.11353093, 0.4383859 , 0.69209206],
        ...,
        [0.9914292 , 0.3724566 , 0.49955344],
        [0.8863649 , 0.58780026, 0.07404406],
        [0.9344226 , 0.37633014, 0.11465928]],

       [[0.16157779, 0.49504828, 0.7726422 ],
        [0.1339223 , 0.52195716, 0.5970783 ],
        [0.13508539, 0.4649422 , 0.9064761 ],
        ...,
        [0.98644066, 0.39313298, 0.7538413 ],
        [0.9872133 , 0.5835457 , 0.13631101],
        [0.99057573, 0.39336178, 0.25916442]],

       [[0.13803118, 0.49037406, 0.8082783 ],
        [0.110451  , 0.51700175, 0.81742144],
        [0.11006087, 0.46295518, 0.8231002 ],
        ...,
        [0.96659446, 0.38554686, 0.8614533 ],
        [0.9427575 , 0.6143918 , 0.17849518],
        [0.93147963, 0.39665017, 0.12604861]],

       ...,

       [[0.09697509, 0.5187658 , 0.62441576],
        [0.0836304 , 0.534857  , 0.57100445],
        [0.08511373, 0

### One hot encoded feature Data 

In [73]:
label_data = np.array([ 0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,
        4,  4,  4,  4,  4,  4,  4,  4,  4,  5,  5,  5,  5,  5,  5,  5,  5,
        5,  5,  5,  5,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  6,  7,
        7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,  8,  8,
        8,  8,  8,  8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 11, 11, 11, 11,
       11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12, 12, 12, 12, 12, 12,
       12, 12, 12, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 13, 14, 14,
       14, 14, 14, 14, 14, 14, 14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15,
       15, 15, 15, 15, 15])

label_data_hot = keras.utils.to_categorical(label_data, num_classes=16)

label_data_hot

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 1.]])

### Neural Networks - Multilayered 

In [116]:
X_train, X_test, y_train, y_test = train_test_split(keypoints_array, label_data_hot, test_size=0.2, random_state=42)

model = Sequential()
model.add(Flatten(input_shape = (17,3)))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=512, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=label_data_hot.shape[1], activation='softmax'))

model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=300, batch_size=55)

Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.0713 - loss: 2.7816 
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.1889 - loss: 2.6557
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.2525 - loss: 2.5706
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3039 - loss: 2.4416
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3374 - loss: 2.3090
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.3218 - loss: 2.1488
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3509 - loss: 2.0127
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4435 - loss: 1.8846
Epoch 9/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.3490 - loss: 1.8195
Epoch 10/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.4315 - loss: 1.7752
Epoch 11/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5246 - loss: 1.6210
Epoch 12/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.5362 - l

In [117]:
loss, accuracy = model.evaluate(X_test, y_test)

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7799 - loss: 0.9008 
